In [1]:
%pip install rasterio 



Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install flask 


Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pyngrok 


Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install numpy 

Note: you may need to restart the kernel to use updated packages.


In [5]:
import rasterio
from flask import Flask, send_from_directory, render_template
import threading
import os


In [6]:
# Path to the GeoTIFF file
filepath = "/Users/haritshah/Downloads/Mag Anom UTM18N NAD83.tif"  # Replace with your GeoTIFF file

# Inspect the GeoTIFF file
with rasterio.open(filepath) as src:
    print(f"Number of Bands: {src.count}")
    print(f"CRS: {src.crs}")
    print(f"Bounds: {src.bounds}")
    print(f"Data Type: {src.dtypes[0]}")

    # Print band statistics
    for i in range(1, src.count + 1):
        print(f"Band {i} Min/Max: {src.read(i).min()}/{src.read(i).max()}")


Number of Bands: 3
CRS: EPSG:26918
Bounds: BoundingBox(left=222952.84713583408, bottom=4182614.9920050376, right=260781.5462814341, top=4220180.493370371)
Data Type: uint8
Band 1 Min/Max: 0/255
Band 2 Min/Max: 0/255
Band 3 Min/Max: 0/255


In [7]:
%pip install gdal

from osgeo import gdal

# Reproject the GeoTIFF to WGS84 (EPSG:4326) using GDAL
input_file = filepath
output_file = os.path.join(os.path.expanduser("~/Desktop"), "FAA_UTM18N_NAD83_WGS84.tif")

gdal.Warp(
    output_file,
    input_file,
    dstSRS="EPSG:4326"
)
print(f"Reprojected GeoTIFF saved to {output_file}")


Note: you may need to restart the kernel to use updated packages.
Reprojected GeoTIFF saved to /Users/haritshah/Desktop/FAA_UTM18N_NAD83_WGS84.tif


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [8]:
import tempfile

# Define directories for Flask server
BASE_DIR = tempfile.gettempdir()
STATIC_DIR = os.path.join(BASE_DIR, "static")
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")

# Create directories if they don't exist
os.makedirs(STATIC_DIR, exist_ok=True)
os.makedirs(TEMPLATES_DIR, exist_ok=True)

# Move GeoTIFF to static folder
os.rename(output_file, os.path.join(STATIC_DIR, os.path.basename(output_file)))

# Flask App
app = Flask(__name__, static_folder=STATIC_DIR, template_folder=TEMPLATES_DIR)

# Serve the map HTML
@app.route("/")
def index():
    return render_template("index.html")

# Serve GeoTIFF files
@app.route("/geotiff/<path:filename>")
def geotiff(filename):
    return send_from_directory(STATIC_DIR, filename)

# Function to run Flask server
def run_flask():
    app.run(port=5000)


In [9]:
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>GeoTIFF in Leaflet</title>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
    <link rel="stylesheet" href="https://unpkg.com/leaflet-control-scale" />
    <link rel="stylesheet" href="https://unpkg.com/leaflet-minimap/dist/Control.MiniMap.min.css" />
    <script src="https://unpkg.com/leaflet"></script>
    <script src="https://unpkg.com/georaster"></script>
    <script src="https://unpkg.com/georaster-layer-for-leaflet"></script>
    <script src="https://unpkg.com/leaflet-control-scale"></script>
    <script src="https://unpkg.com/leaflet-minimap"></script>
    <style>
        .legend {
            line-height: 1.5em;
            padding: 6px 8px;
            font: 14px/16px Arial, Helvetica, sans-serif;
            box-shadow: 0 0 15px rgba(0, 0, 0, 0.2);
            border-radius: 5px;
            background: white;
        }
        .legend i {
            width: 18px;
            height: 18px;
            float: left;
            margin-right: 8px;
        }
        .legend .blue { background: blue; }
        .legend .green { background: green; }
        .legend .yellow { background: yellow; }
        .legend .orange { background: orange; }
        .legend .red-magenta { background: linear-gradient(to right, red, magenta); }
    </style>
</head>
<body>
    <div id="map" style="width: 100%; height: 600px;"></div>
    <script>
        const map = L.map('map').setView([38.0, -77.0], 8);

        // Add base map
        const baseLayer = L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
            maxZoom: 19,
            attribution: '© OpenStreetMap contributors'
        }).addTo(map);

        // Add scale bar
        L.control.scale().addTo(map);

        // Load GeoTIFF
        fetch('/geotiff/FAA_UTM18N_NAD83_WGS84.tif')  // Use the reprojected file
            .then(response => response.arrayBuffer())
            .then(arrayBuffer => {
                parseGeoraster(arrayBuffer).then(georaster => {
                    const layer = new GeoRasterLayer({
                        georaster: georaster,
                        opacity: 0.8,
                        resolution: 128  // Adjust the resolution to make pixels smaller
                    });
                    layer.addTo(map);
                    map.fitBounds(layer.getBounds());

                    // Add layer control
                    const baseMaps = {
                        "OpenStreetMap": baseLayer
                    };
                    const overlayMaps = {
                        "GeoTIFF Layer": layer
                    };
                    L.control.layers(baseMaps, overlayMaps).addTo(map);
                });
            });

        // Add legend
        const legend = L.control({ position: 'bottomright' });
        legend.onAdd = function (map) {
            const div = L.DomUtil.create('div', 'legend');
            div.innerHTML += '<b>Legend</b><br>';
            div.innerHTML += '<i class="blue"></i> 46 nT (Blue)<br>';
            div.innerHTML += '<i class="green"></i> 98-122 nT (Green)<br>';
            div.innerHTML += '<i class="yellow"></i> 154 nT (Yellow)<br>';
            div.innerHTML += '<i class="orange"></i> 194 nT (Orange)<br>';
            div.innerHTML += '<i class="red-magenta"></i> 267 nT (Red to Magenta)<br>';
            div.innerHTML += '<small>nT: nanoTesla</small>';
            return div;
        };
        legend.addTo(map);

        // Add popup on click
        map.on('click', function (e) {
            L.popup()
                .setLatLng(e.latlng)
                .setContent("You clicked the map at " + e.latlng.toString())
                .openOn(map);
        });

        // Add mini map
        const miniMapLayer = L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
            maxZoom: 19,
            attribution: '© OpenStreetMap contributors'
        });
        const miniMap = new L.Control.MiniMap(miniMapLayer, {
            toggleDisplay: true,
            minimized: false
        }).addTo(map);
    </script>
</body>
</html>
"""

with open(os.path.join(TEMPLATES_DIR, "index.html"), "w") as f:
    f.write(html_content)


In [10]:
app = Flask(__name__, static_folder=STATIC_DIR, template_folder=TEMPLATES_DIR)

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/geotiff/<path:filename>")
def geotiff(filename):
    return send_from_directory(STATIC_DIR, filename)

# Start Flask server in a background thread
def run_flask():
    app.run(port=5000)

thread = threading.Thread(target=run_flask)
thread.start()

print("Flask server is running! Open http://127.0.0.1:5000 in your browser.")


Flask server is running! Open http://127.0.0.1:5000 in your browser.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Nov/2024 00:28:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2024 00:28:21] "GET /geotiff/FAA_UTM18N_NAD83_WGS84.tif HTTP/1.1" 200 -
